# 결측치 정제

## #01. 준비과정

### [1] 패키지 참조

In [20]:
from pandas import read_excel, DataFrame
from sklearn.impute import SimpleImputer
import numpy as np

### [2] 데이터 가져오기

In [21]:
# origin = read_excel("https://data.hossam.kr/pydata/ref_sample.xlsx", index_col="name")
origin = read_excel('./res/ref_sample.xlsx',index_col='name')
origin

,kor,eng,math,sic
name,,,,
철수,98.0,77,88.0,64.0
영희,88.0,120,62.0,72.0
민철,NaN,70,83.0,79.0
수현,63.0,60,31.0,71.0
호영,75.0,50,90.0,NaN
영호,80.0,88,91.0,72.0
용식,82.0,88,NaN,90.0
나영,90.0,92,81.0,NaN
석영,91.0,90,89.0,80.0


## #02. 결측치 확인하기

### [1] 결측치 여부 확인

각 열에 대해 결측치가 아닐 경우 `False`, 결측치는 `True`로 표시됨

`isna()` 함수도 같은 기능

In [22]:
empty = origin.isnull() # isna()도 동일한 효과를 갖는 메서드임
empty

,kor,eng,math,sic
name,,,,
철수,False,False,False,False
영희,False,False,False,False
민철,True,False,False,False
수현,False,False,False,False
호영,False,False,False,True
영호,False,False,False,False
용식,False,False,True,False
나영,False,False,False,True
석영,False,False,False,False


### [2] 각 열별로 결측치의 수를 확인

`isnull()` 혹은 `isna()`의 결과에 대한 합계를 구한다.

`True=1`, `False=0`

In [23]:
empty.sum() # 각 열별 결측치수

kor     1
eng     0
math    1
sic     2
dtype: int64

## #03. 결측치 처리

### [1] 결측치 소거

#### (1) 행 단위 삭제

결측치가 있는 모든 행 삭제 (원본은 변화 없음, 삭제 결과 리턴됨)

`inplace=True`를 적용할 경우 원본에 즉시 반영되고 리턴값 없음

In [24]:
na1 = origin.dropna()
na1

,kor,eng,math,sic
name,,,,
철수,98.0,77,88.0,64.0
영희,88.0,120,62.0,72.0
수현,63.0,60,31.0,71.0
영호,80.0,88,91.0,72.0
석영,91.0,90,89.0,80.0


#### (2) 열 단위 삭제

`dropna()` 메서드에 `axis=1` 파라미터를 적용한다. (`기본값=0`)

In [25]:
na2 = origin.dropna(axis=1)
na2

,eng
name,
철수,77
영희,120
민철,70
수현,60
호영,50
영호,88
용식,88
나영,92
석영,90


## #04. 결측치 대체

### [1] 고정값으로 대체

모든 결측치를 동일한 값으로 교체

원본은 변화 없음, 삭제결과 리턴됨

`fillna` 메서드를 이용함. value라는 파라미터에 숫자나 문자를 입력하면 그대로 결측치를 대체하고 딕셔너리형태로 주어지면 각각의 열에 대해 원하는 값으로 변경이 된다.

method라는 파라미터로 ffill을 주는 경우 결측값이 위값과 동일하게 설정된다. ( 가장 위의값이 결측치인 경우 대체 불가), bfill을 주는 경우 결측값이 아래값과 동일하게 설정된다.

`inplace=True`를 적용할 경우 원본에 즉시 반영되고 리턴값 없음

In [26]:
re_df = origin.fillna(value=50)
re_df

,kor,eng,math,sic
name,,,,
철수,98.0,77,88.0,64.0
영희,88.0,120,62.0,72.0
민철,50.0,70,83.0,79.0
수현,63.0,60,31.0,71.0
호영,75.0,50,90.0,50.0
영호,80.0,88,91.0,72.0
용식,82.0,88,50.0,90.0
나영,90.0,92,81.0,50.0
석영,91.0,90,89.0,80.0


In [31]:
re_df1 = origin.fillna(value={'kor':0,'eng':10,'math':20}) # 딕셔너리에 주어지지 않은 열은(키는) 그대로 결측치로 남아있는다
re_df1

,kor,eng,math,sic
name,,,,
철수,98.0,77,88.0,64.0
영희,88.0,120,62.0,72.0
민철,0.0,70,83.0,79.0
수현,63.0,60,31.0,71.0
호영,75.0,50,90.0,NaN
영호,80.0,88,91.0,72.0
용식,82.0,88,20.0,90.0
나영,90.0,92,81.0,NaN
석영,91.0,90,89.0,80.0


### [2] 통계적 값으로 대체

#### (1) 결측치를 정제할 규칙을 담고 있는 객체 생성

각 열단위로 평균`(strategy='mean')`을 결측치`(missing_values)`에 지정

strategy 옵션 : mean=평균, median=중앙값, most_frequent: 최빈값(가장 많이 관측되는 수), constant: fill_value라는 파라미터로 임의의값을 줄 수 있음(str도 가능) fill_value의 default값은 0

> 이 객체의 생성이 결측치를 즉시 처리했다는 것을 의미하는 것은 아니다.

In [28]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean') # missing_values=np.nan, strategy='mean' 가 기본값임
imr

SimpleImputer()

#### (2) 생성된 규칙을 적용

결측치가 정제된 2차원 배열로 리턴되므로 다시 데이터프레임으로 재구성할 필요가 있다.

In [29]:
df_imr = imr.fit_transform(origin.values)
df_imr

array([[ 98.        ,  77.        ,  88.        ,  64.        ],
       [ 88.        , 120.        ,  62.        ,  72.        ],
       [ 83.375     ,  70.        ,  83.        ,  79.        ],
       [ 63.        ,  60.        ,  31.        ,  71.        ],
       [ 75.        ,  50.        ,  90.        ,  75.42857143],
       [ 80.        ,  88.        ,  91.        ,  72.        ],
       [ 82.        ,  88.        ,  76.875     ,  90.        ],
       [ 90.        ,  92.        ,  81.        ,  75.42857143],
       [ 91.        ,  90.        ,  89.        ,  80.        ]])

In [30]:
re_df2 = DataFrame(df_imr, index=origin.index, columns=origin.columns)
re_df2

,kor,eng,math,sic
name,,,,
철수,98.000,77.0,88.000,64.000000
영희,88.000,120.0,62.000,72.000000
민철,83.375,70.0,83.000,79.000000
수현,63.000,60.0,31.000,71.000000
호영,75.000,50.0,90.000,75.428571
영호,80.000,88.0,91.000,72.000000
용식,82.000,88.0,76.875,90.000000
나영,90.000,92.0,81.000,75.428571
석영,91.000,90.0,89.000,80.000000
